In [2]:
import modish
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pybedtools

bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages'
! mkdir $bed_folder


DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/exon2.bed'.format(DIR)
exon_bed = pybedtools.BedTool(exon_bedfile)

from modish.visualize import MODALITY_TO_COLOR, MODALITY_ORDER, MODALITY_PALETTE
modality_order = MODALITY_ORDER

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages'

import flotilla

flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'
study = flotilla.embark('singlecell_pnm_figure4_voyages', flotilla_dir=flotilla_dir)

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages': File exists
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2015-12-04 11:34:36	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure4_voyages/datapackage.json
2015-12-04 11:34:36	Parsing datapackage to create a Study object
2015-12-04 11:35:29	Initializing Study
2015-12-04 11:35:29	Initializing Predictor configuration manager for Study
2015-12-04 11:35:29	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2015-12-04 11:35:29	Added ExtraTreesClassifier to default predictors
2015-12-04 11:35:29	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2015-12-04 11:35:29	Added ExtraTreesRegressor to default predictors
2015-12-04 11:35:29	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.Gr

/home/obotvinnik/workspace-git/flotilla/flotilla/__init__.py:55: DtypeWarning: Columns (11,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


## Filter on big voyages

In [3]:
voyages_filtered = study.supplemental.voyages.groupby(['transition', 'direction'], as_index=False, group_keys=False).apply(
    lambda x: x.loc[x.voyage_distance > x.voyage_distance.quantile(0.8)])

# voyages_filtered = study.supplemental.voyages.groupby(['transition', 'direction']).apply(
#     lambda x: x.loc[(x[delta_cols].abs() > (x[delta_cols].abs().mean() + 1*x[delta_cols].abs().std())).any(axis=1)])
print voyages_filtered.groupby('transition').size()

transition
NPC-MN      476
iPSC-MN     475
iPSC-NPC    917
dtype: int64


In [ ]:
homer_dir = '{}/homer'.format(bed_folder)
!mkdir -p $homer_dir


In [5]:
study.supplemental.voyages.head()

,event_id,$\Delta x$,$\Delta y$,voyage_distance,group1,group2,direction,transition
0,exon:chr10:102114184-102114389:+@exon:chr10:10...,0.053571,-0.054198,0.076206,iPSC,NPC,$\searrow$,iPSC-NPC
1,exon:chr10:102286732-102286831:-@exon:chr10:10...,0.000000,0.000000,0.000000,iPSC,NPC,$\swarrow$,iPSC-NPC
2,exon:chr10:102747070-102747190:-@exon:chr10:10...,0.004480,-0.004480,0.006336,iPSC,NPC,$\searrow$,iPSC-NPC
3,exon:chr10:103348089-103348157:+@exon:chr10:10...,0.000000,-0.027778,0.027778,iPSC,NPC,$\swarrow$,iPSC-NPC
4,exon:chr10:103360960-103361093:+@exon:chr10:10...,-0.056306,0.055556,0.079100,iPSC,NPC,$\nwarrow$,iPSC-NPC


In [ ]:
import os
import pybedtools

DIRECTIONS = 'upstream', 'downstream'

commands = []

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {} -noweight'.format(n_processors)
                                                                                
primate_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPrimates.bed'
primate = pybedtools.BedTool(primate_filename)
placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
placental = pybedtools.BedTool(placental_filename)
conserved_regions = {'primate': primate, 'placental': placental}

genome = 'hg19'

nt = 400

def filter_and_save(original_bed, names, filename):
    filtered_bed = original_bed.filter(lambda x: x.name in names)
    return filtered_bed.saveas(filename)

transition_grouped = study.supplemental.voyages.groupby(['transition'])

for transition, transition_df in voyages_filtered.groupby('transition'):
    transition_events = set(transition_df.event_id)
    for direction, direction_df in transition_df.groupby('direction'):
    
        foreground_events = set(direction_df.event_id)
        format_args = bed_folder, transition, direction.strip('$\\')
    #     print format_args
        foreground_filename = '{}/exon2_{}_{}_foreground.bed'.format(*format_args)
        filter_and_save(exon_bed, foreground_events, foreground_filename)
        ! wc -l $filtered_filename

        background_events = transition_events - foreground_events
        background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
        filter_and_save(exon_bed, background_events, background_filename)
        ! wc -l $filtered_filename
    
        pair = pyhomer.ForegroundBackgroundPair(foreground_filename, background_filename, genome='hg19')
        
        for clade, conservation_bed in conserved_regions.items():
            conserved_pair = pair.intersect(conservation_bed, clade)
            print '\n', conserved_pair
            suffix = os.path.basename(conserved_pair.foreground.fn).replace('_foreground.bed', '')
            out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
            
            # Add search for motifs in conserved exon bodies
            command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
            commands.append(command)
        
        for direction in DIRECTIONS:
            intron_pair = pair.flanking_intron('downstream', 'hg19', 400)
            print '\n', intron_pair
            suffix = os.path.basename(intron_pair.foreground.fn).replace('_foreground.bed', '')
            out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)

            for clade, conservation_bed in conserved_regions.items():
                conserved_introns = intron_pair.intersect(conservation_bed, clade)
                print '\n', conserved_introns
                suffix = os.path.basename(conserved_introns.foreground.fn).replace('_foreground.bed', '')
                out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
                
                # Add search for motifs in conserved introns
                command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
                commands.append(command)
                                                                                
# for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
#     background_events = set(phenotype_df.event_name)
#     for modality, modality_df in phenotype_df.groupby('modality'):
#         event_names = set(modality_df.event_name)
#         format_args = bed_folder, phenotype, modality
#         foreground_bed = exon_bed.filter(lambda x: x.name in event_names)
#         foreground_filename = '{}/exon2_{}_{}_foreground.bed'.format(*format_args)
#         foreground = pybedtools.BedTool(foreground_bed.saveas(foreground_filename))

#         background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
#         background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
#         background = pybedtools.BedTool(background_bed.saveas(background_filename))
        
#         pair = pyhomer.ForegroundBackgroundPair(foreground, background)
#         print '\n', pair 
        
#         suffix = os.path.basename(pair.foreground.fn).replace('_foreground.bed', '')
#         out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
# #         command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
# #         commands.append(command)

#         beds = {'background': background_bed, 'foreground': foreground_bed}
        
#         for clade, conservation_bed in conserved_regions.items():
#             conserved_pair = pair.intersect(conservation_bed, clade)
#             print '\n', conserved_pair
#             suffix = os.path.basename(conserved_pair.foreground.fn).replace('_foreground.bed', '')
#             out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
#             command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
#             commands.append(command)
        
#         for direction in DIRECTIONS:
#             intron_pair = pair.flanking_intron('downstream', 'hg19', 400)
#             print '\n', intron_pair
#             suffix = os.path.basename(intron_pair.foreground.fn).replace('_foreground.bed', '')
#             out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
# #             command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
# #             commands.append(command)

#             for clade, conservation_bed in conserved_regions.items():
#                 conserved_introns = intron_pair.intersect(conservation_bed, clade)
#                 print '\n', conserved_introns
#                 suffix = os.path.basename(conserved_introns.foreground.fn).replace('_foreground.bed', '')
#                 out_dir = '{base_dir}/{suffix}'.format(base_dir=homer_dir, suffix=suffix)
#                 command = pair.homer(homer_flags, findMotifsGenome, out_dir=out_dir)
#                 commands.append(command)

## Make bed files

In [ ]:
%%time

def filter_and_save(original_bed, names, filename):
    filtered_bed = original_bed.filter(lambda x: x.name in names)
    return filtered_bed.saveas(filename)

transition_grouped = study.supplemental.voyages.groupby(['transition'])

for (transition, direction), df in voyages_filtered.groupby(['transition', 'direction']):
    
    foreground_events = set(df.event_id)
    format_args = bed_folder, transition, direction.strip('$\\')
#     print format_args
    filtered_filename = '{}/exon2_{}_{}.bed'.format(*format_args)
    filter_and_save(exon_bed, foreground_events, filtered_filename)
    ! wc -l $filtered_filename
    
    background_df = transition_grouped.get_group(transition)
    background_events = set(background_df.event_id) - foreground_events
    filtered_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
    filter_and_save(exon_bed, background_events, filtered_filename)
    ! wc -l $filtered_filename


In [ ]:
homer_dir = '{}/homer'.format(bed_folder)
!mkdir -p $homer_dir

### Get upstream/downstream sequences

In [ ]:
import os
import glob
import pybedtools

nt = 400

directions = 'upstream', 'downstream'

for bedfile in glob.iglob('{}/exon2*.bed'.format(bed_folder)):
    if 'stream' in bedfile:
        continue
    
    basename = os.path.basename(bedfile)
    print basename
    prefix = basename.split('.bed')[0]
    bed = pybedtools.BedTool(bedfile)
    
    for direction in directions:
        # Get downstream intron
        if direction == 'downstream':
            intron = bed.flank(l=0, r=nt, s=True, g=pybedtools.chromsizes('hg19'))
        elif direction == 'upstream':
            intron = bed.flank(l=nt, r=0, s=True, g=pybedtools.chromsizes('hg19'))
            
        # Saved every exon that was exactly upstream or downstream of a junction,
        # So when taking the flanking sequence, there's a lot of repetition
        unique = pybedtools.BedTool(list(set(x for x in intron)))


        print '\t', len(intron), len(unique)
        saveas = '{}/{}_{}{}.bed'.format(bed_folder, prefix, direction, nt)
        print '\t', saveas
        unique.saveas(saveas)
        ! wc -l $saveas

In [6]:
cd $homer_dir

/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/homer


In [7]:

import os
import fileinput

from gscripts.qtools import Submitter
import glob
import itertools

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {}'.format(n_processors)

commands = []


for bedfile in glob.iglob('{}/exon2*.bed'.format(bed_folder)):
    
    basename = os.path.basename(bedfile)
    if 'background' in basename:
        continue
    
    prefix = basename.split('.')[0]
    transition = prefix.split('_')[1]
    direction = prefix.split('_')[2]
#     print modality
    
    background = bedfile.replace(direction, direction + '_background')
    
    out_dir = '{}/{}'.format(homer_dir, basename.replace('.bed', ''))
    if os.path.exists('{}/homerResults.html'):
        print 'Already finished {}, moving on'.format(basename)
        continue
    command = '{} {} hg19 {} -bg {} {}'.format(
        findMotifsGenome, bedfile, out_dir, background, homer_flags)
    print '\n', command
    commands.append(command)
#     ! $command
jobname = 'homer'
sub = Submitter(commands, jobname, walltime='0:30:00', queue='home-scrm',
               ppn=n_processors, write_and_submit=True, array=True,
               out_filename='{}/{}.out'.format(homer_dir, jobname),
               err_filename='{}/{}.err'.format(homer_dir, jobname),)

running 60 tasks as an array-job.
job ID: 3798110



/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/exon2_iPSC-NPC_swarrow.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/homer/exon2_iPSC-NPC_swarrow -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/exon2_iPSC-NPC_swarrow_background.bed -rna -len 4,5,6 -mset vertebrates -mis 0 -p 4

/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/exon2_NPC-MN_nwarrow_upstream200.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/homer/exon2_NPC-MN_nwarrow_upstream200 -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/exon2_NPC-MN_nwarrow_background_upstream200.bed -rna -len 4,5,6 -mset vertebrates -mis 0 -p 4

/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/exon2_NPC-MN_swarrow_downstream400.bed hg19 /projects/ps-yeolab/obotv

In [8]:
prefix

'exon2_iPSC-NPC_nwarrow_downstream200'